In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [2]:
import yfinance as yf
import numpy as np
import pandas as pd

def monte_carlo_simulation(S0, K, T, r, sigma, M, I):
    dt = T / M
    Z = np.random.standard_normal((M, I))
    S = np.zeros((M + 1, I))
    S[0] = S0
    for t in range(1, M + 1):
        S[t] = S[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z[t - 1])

    payoff = np.maximum(S[-1] - K, 0)
    option_price = np.exp(-r * T) * np.mean(payoff)
    return option_price

data = yf.download('AAPL', start='2023-01-01', end='2024-01-01')

S0 = data['Close'][-1]  # Last closing price
K = 180.0               # Strike price
T = 30 / 365            # Time to maturity (30 days)
r = 0.05                # Risk-free rate (5%)
sigma = 0.25            # Volatility (25%)
M = 100                 # Number of time steps
I = 100000              # Number of simulations


entry_dates = []
exit_dates = []
profits = []


for i in range(len(data) - 1):
    S0 = data['Close'][i]
    entry_date = data.index[i]
    
    monte_carlo_price = monte_carlo_simulation(S0, K, T, r, sigma, M, I)
    
    if monte_carlo_price < 2.0:
        entry_dates.append(entry_date)
        
        exit_date = data.index[min(i + 10, len(data) - 1)]
        S1 = data['Close'][min(i + 10, len(data) - 1)]
        
        exit_price = monte_carlo_simulation(S1, K, T, r, sigma, M, I)
        profits.append(exit_price - monte_carlo_price)
        exit_dates.append(exit_date)

trade_details_monte_carlo = pd.DataFrame({
    'Entry Date': entry_dates,
    'Exit Date': exit_dates,
    'Profit': profits
})

trade_details_monte_carlo.to_csv('monte_carlo_trade_details.csv', index=False)

print("Monte Carlo trade details saved to monte_carlo_trade_details.csv")


[*********************100%%**********************]  1 of 1 completed
C:\Users\Amol Pagare\AppData\Local\Temp\ipykernel_29140\2935318811.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S0 = data['Close'][-1]  # Last closing price
C:\Users\Amol Pagare\AppData\Local\Temp\ipykernel_29140\2935318811.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S0 = data['Close'][i]
C:\Users\Amol Pagare\AppData\Local\Temp\ipykernel_29140\2935318811.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame beha

Monte Carlo trade details saved to monte_carlo_trade_details.csv


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf

def binomial_model(S0, K, T, r, sigma, N, option_type='call'):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))  # Up factor
    d = 1 / u                        # Down factor
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability

    ST = np.zeros(N + 1)
    for i in range(N + 1):
        ST[i] = S0 * (u ** (N - i)) * (d ** i)

    if option_type == 'call':
        option = np.maximum(ST - K, 0)
    elif option_type == 'put':
        option = np.maximum(K - ST, 0)

    for j in range(N - 1, -1, -1):
        for i in range(j + 1):
            option[i] = np.exp(-r * dt) * (p * option[i] + (1 - p) * option[i + 1])

    return option[0]


data = yf.download('AAPL', start='2023-01-01', end='2024-01-01')


K = 180.0               # Strike price
T = 30 / 365            # Time to maturity (30 days)
r = 0.05                # Risk-free rate (5%)
sigma = 0.25            # Volatility (25%)
N = 100                 # Number of time steps


entry_dates = []
exit_dates = []
profits = []


for i in range(len(data) - 1):
    S0 = data['Close'][i]
    entry_date = data.index[i]
    
    binomial_price = binomial_model(S0, K, T, r, sigma, N, option_type='call')
    
    if binomial_price < 2.0:
        entry_dates.append(entry_date)
        
        exit_date = data.index[min(i + 10, len(data) - 1)]
        S1 = data['Close'][min(i + 10, len(data) - 1)]
        
        exit_price = binomial_model(S1, K, T, r, sigma, N, option_type='call')
        profits.append(exit_price - binomial_price)
        exit_dates.append(exit_date)


trade_details_binomial = pd.DataFrame({
    'Entry Date': entry_dates,
    'Exit Date': exit_dates,
    'Profit': profits
})

trade_details_binomial.to_csv('binomial_trade_details.csv', index=False)

print("Binomial trade details saved to binomial_trade_details.csv")


[*********************100%%**********************]  1 of 1 completed
C:\Users\Amol Pagare\AppData\Local\Temp\ipykernel_29140\345271170.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S0 = data['Close'][i]
C:\Users\Amol Pagare\AppData\Local\Temp\ipykernel_29140\345271170.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S1 = data['Close'][min(i + 10, len(data) - 1)]


Binomial trade details saved to binomial_trade_details.csv


In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm

def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return price

data = yf.download('AAPL', start='2023-01-01', end='2024-01-01')

K = 180.0               # Strike price
T = 30 / 365            # Time to maturity (30 days)
r = 0.05                # Risk-free rate (5%)
sigma = 0.25            # Volatility (25%)

entry_dates = []
exit_dates = []
profits = []

for i in range(len(data) - 1):
    S0 = data['Close'][i]
    entry_date = data.index[i]
    
    bs_price = black_scholes(S0, K, T, r, sigma, option_type='call')
    
    if bs_price < 2.0:
        entry_dates.append(entry_date)
        
        exit_date = data.index[min(i + 10, len(data) - 1)]
        S1 = data['Close'][min(i + 10, len(data) - 1)]
        
        exit_price = black_scholes(S1, K, T, r, sigma, option_type='call')
        profits.append(exit_price - bs_price)
        exit_dates.append(exit_date)

trade_details_black_scholes = pd.DataFrame({
    'Entry Date': entry_dates,
    'Exit Date': exit_dates,
    'Profit': profits
})

trade_details_black_scholes.to_csv('black_scholes_trade_details.csv', index=False)

print("Black-Scholes trade details saved to black_scholes_trade_details.csv")


[*********************100%%**********************]  1 of 1 completed

Black-Scholes trade details saved to black_scholes_trade_details.csv



C:\Users\Amol Pagare\AppData\Local\Temp\ipykernel_29140\2168548711.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S0 = data['Close'][i]
C:\Users\Amol Pagare\AppData\Local\Temp\ipykernel_29140\2168548711.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S1 = data['Close'][min(i + 10, len(data) - 1)]
